In [2]:
#Import necessary Libraries
import pandas as pd
import numpy as np

import string
import re
import os

import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import TextVectorization
from tensorflow import keras
from tensorflow.keras import layers

from params import *


In [3]:
my_df = pd.read_csv("Dataset/data.csv")
print("**********************")
my_df.head()



**********************


,english,spanish
0,Go.,Ve.
1,Go.,Vete.
2,Go.,Vaya.
3,Go.,Váyase.
4,Hi.,Hola.


In [4]:
my_df.info()
print("**********************")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118964 entries, 0 to 118963
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   english  118964 non-null  object
 1   spanish  118964 non-null  object
dtypes: object(2)
memory usage: 1.8+ MB
**********************


In [5]:
# This line of code modifies the "spanish" column of the DataFrame `my_df` by applying a lambda function to each item in the column..

my_df["spanish"] = my_df["spanish"].apply(lambda item: Get_Params.begin_token + " " + item + " " + Get_Params.end_token)

In [6]:
missing_values = my_df.isnull().sum()

print("Missing Values:")
print(missing_values)

my_df.head()

Missing Values:
english    0
spanish    0
dtype: int64


,english,spanish
0,Go.,[start] Ve. [end]
1,Go.,[start] Vete. [end]
2,Go.,[start] Vaya. [end]
3,Go.,[start] Váyase. [end]
4,Hi.,[start] Hola. [end]
